In [54]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import math,numpy

In [72]:
us_counties_file = "live/us-counties.csv"
counties_df = pd.read_csv(us_counties_file)
counties_df.head()

,date,county,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths
0,2020-08-01,Autauga,Alabama,1001.0,1015,21.0,972.0,20.0,43.0,1.0
1,2020-08-01,Baldwin,Alabama,1003.0,3101,22.0,3056.0,21.0,45.0,1.0
2,2020-08-01,Barbour,Alabama,1005.0,598,5.0,550.0,5.0,48.0,0.0
3,2020-08-01,Bibb,Alabama,1007.0,363,2.0,355.0,2.0,8.0,0.0
4,2020-08-01,Blount,Alabama,1009.0,767,3.0,685.0,3.0,82.0,0.0


### Transform County DataFrame ###

In [82]:
# Create a filtered dataframe from specific columns
counties_cols = ["county", "state", "fips","confirmed_cases","confirmed_deaths"]
counties_transformed= counties_df[counties_cols].copy()

# Rename the column headers
counties_transformed = counties_transformed.rename(columns={"fips": "COUNTYFP",
                                                              "county": "COUNTY",
                                                              "state": "STATE",
                                                                "confirmed_cases":"CONFIRMED_CASES",
                                                                "confirmed_deaths":"CONFIRMED_DEATHS"})

counties_transformed["COUNTYFP"] = counties_transformed["COUNTYFP"].apply(np.int64)
counties_transformed
counties_transformed.set_index("COUNTYFP", inplace=True)

counties_transformed.head()

,COUNTY,STATE,CONFIRMED_CASES,CONFIRMED_DEATHS
COUNTYFP,,,,
1001,Autauga,Alabama,972.0,20.0
1003,Baldwin,Alabama,3056.0,21.0
1005,Barbour,Alabama,550.0,5.0
1007,Bibb,Alabama,355.0,2.0
1009,Blount,Alabama,685.0,3.0


### Transform Mask-Usage Dataframe###

In [83]:
mask_use_file = "mask-use/mask-use-by-county.csv"
mask_use_df = pd.read_csv(mask_use_file)
mask_use_df.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [84]:
mask_use_df.set_index("COUNTYFP", inplace=True)
mask_use_df.head()

,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
COUNTYFP,,,,,
1001,0.053,0.074,0.134,0.295,0.444
1003,0.083,0.059,0.098,0.323,0.436
1005,0.067,0.121,0.120,0.201,0.491
1007,0.020,0.034,0.096,0.278,0.572
1009,0.053,0.114,0.180,0.194,0.459


<!-- Create database connection -->

### Create Database Connection ###

In [86]:
connection_string = "postgres:postgres@localhost:5432/covidmask_db"
engine = create_engine(f'postgresql://{connection_string}')

In [87]:
# Confirm tables
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  database "mask_db" does not exist

(Background on this error at: http://sqlalche.me/e/e3q8)

### Load DataFrames into Database ###

In [88]:
counties_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)

OperationalError: (psycopg2.OperationalError) FATAL:  database "mask_db" does not exist

(Background on this error at: http://sqlalche.me/e/e3q8)

In [89]:
mask_use_df.to_sql(name='mask', con=engine, if_exists='append', index=True)

OperationalError: (psycopg2.OperationalError) FATAL:  database "mask_db" does not exist

(Background on this error at: http://sqlalche.me/e/e3q8)